In [556]:
import pandas as pd
import numpy as np
import rapidfuzz

In [557]:
state='NJ'
path = r"C:\Users\david\OneDrive\Documents\GitHub\SPEOC-pt-1\Data\Post1790\NJ\NJ_3_percent_stock_T698_R1_R2.xlsx"
occs_and_debt = pd.read_excel(path,
                             header=11, usecols='G, L, M') #occupations plus continental debt for conneticut
print(occs_and_debt)

                  Unnamed: 6  Dollars  Cents
0               Schoolmaster    162.0   51.0
1                  Carpinter     41.0   10.0
2                        NaN     70.0   44.0
3                    Esquire    684.0    7.0
4                        NaN     92.0   42.0
..                       ...      ...    ...
971                    Tamer    430.0   38.0
972                      NaN     47.0   35.0
973                      NaN     19.0   44.0
974                      NaN      9.0   62.0
975  Executor Rebecca Forman    165.0   84.0

[976 rows x 3 columns]


In [558]:
occs_and_debt = occs_and_debt.rename(columns={'Unnamed: 6': 'Occupation'})
occs_and_debt = occs_and_debt.dropna(subset=['Occupation'])
print(occs_and_debt.to_string())

                                      Occupation  Dollars  Cents
0                                   Schoolmaster    162.0   51.0
1                                      Carpinter     41.0   10.0
3                                        Esquire    684.0    7.0
7                                        Esquire    128.0   46.0
9     Guardian of Sons of George Green  Deceased    428.0    NaN
10                                      Senator    5734.0   86.0
11                             Counsellor at Low    942.0   51.0
14                                        Farmer    137.0   94.0
15                                        Farmer   1183.0   74.0
18                                     Carpenter     84.0   98.0
20                           Speaker of Assembly    168.0   52.0
21                                        Farmer    215.0   17.0
24                                        Farmer    429.0   16.0
25                                        Farmer    454.0   83.0
26                       

In [559]:
occurences_dict = occs_and_debt['Occupation'].value_counts().to_dict()
print(occurences_dict)

{'Farmer': 72, 'Esquire': 69, 'Merchant': 51, 'Widow': 37, 'Broker': 12, 'Physician': 10, 'Esq': 9, 'Counseller at Law': 7, 'Inn Keeper': 7, 'Attorney at Law': 6, 'Counsellor at Law': 6, 'Druggist': 5, 'Maiden': 4, 'Blacksmith': 4, 'Taylor': 4, 'County': 3, 'Counseller at Low': 3, 'Merchants': 3, 'Turner': 3, 'Barker': 3, 'Clerk': 3, 'Senator ': 3, 'Executor Rebecca Forman': 2, 'Physian': 2, 'Hunterdon County': 2, 'Late Sheriff': 2, 'School Master': 2, 'Widers': 2, 'Fanner': 2, 'Stage Driver': 2, 'Late of Trenton': 2, 'Schoolmaster': 2, 'Esq Counseller at Law': 2, 'Mason': 2, 'Carpenter': 2, 'Speaker of Assembly': 2, 'Practitioner in Physick': 2, 'Attorney at law': 2, 'Baker': 2, 'Attorney General': 2, 'Hatter': 2, 'Now Peasson, Wife of Job Peasson': 2, 'Cooper': 2, 'Brewer': 2, 'Executor of Thomas Newman': 1, 'Gentleman': 1, 'Executor of Cornelius Van Horne': 1, 'farmer': 1, 'Treasurer': 1, 'Smalley of Cumberland': 1, 'Adm of John Holmes Deceased': 1, 'Executor of John Budd Deceased':

In [560]:
totals = occs_and_debt.groupby('Occupation')[['Dollars', 'Cents']].apply(sum)
totals = totals.reset_index()
print(totals.to_string())

                                     Occupation  Dollars   Cents
0                              (Son of Timothy)    126.0    79.0
1                   Adm of John Holmes Deceased      0.0     0.0
2                Adm of William Bunnet Deceased      3.0    87.0
3             Admin of James McMichael Deceased    132.0    97.0
4        Administrator of John Connets deceased     89.0    25.0
5     Administrators of James Chambers Deceased    125.0    69.0
6       Admistrator of James McMichael Deceased    132.0    97.0
7                                        At Law     49.0    50.0
8                                Attoney at Law    278.0    63.0
9                                 Attony at Low     19.0    75.0
10                             Attorney General   3885.0   108.0
11                              Attorney at Law    201.0   382.0
12                              Attorney at law     16.0    56.0
13                                        Baker   1792.0   116.0
14                       

In [561]:
occs_debt_wo_dups = totals.copy()
#use fuzzy string matching to remove any similar names like merchant and merchents. does not take into account longer titles.
occ_list = occs_debt_wo_dups['Occupation'].tolist()

for occ in occ_list:
    for occ_2 in occ_list:
        if occ != occ_2:
            if rapidfuzz.fuzz.ratio(occ, occ_2) >= 80 and occ != occ_2:
                #transfer amount in row that will be deleted to other row
                row_w_occ_2 = occs_debt_wo_dups.loc[occs_debt_wo_dups['Occupation'] == occ_2] #locate row that contains occupation soon to be deleted
                dollars_in_occ_2 = row_w_occ_2[['Dollars']].sum(axis=1).values #get total dollars
                cents_in_occ_2 = row_w_occ_2[['Cents']].sum(axis=1).values #get total cents
                dollars_in_occ_2 = dollars_in_occ_2[0] #stored in array. need to retrieve value
                cents_in_occ_2 = cents_in_occ_2[0] #stored in array. need to retrieve value  
                occs_debt_wo_dups.loc[(occs_debt_wo_dups.Occupation == occ), 'Dollars'] += dollars_in_occ_2 #add dollars to occupation 1
                occs_debt_wo_dups.loc[(occs_debt_wo_dups.Occupation == occ), 'Cents'] += cents_in_occ_2 #add cents to occupation 1
                 
                #remove value from occurences dictionary and add one to other occupation
                occurences_dict[occs_debt_wo_dups.loc[(occs_debt_wo_dups.Occupation == occ), 'Occupation'].values[0]] += occurences_dict[occs_debt_wo_dups.loc[(occs_debt_wo_dups.Occupation == occ_2), 'Occupation'].values[0]] 
                occurences_dict.pop(occ_2)
                
                #remove row
                occs_debt_wo_dups.drop(occs_debt_wo_dups[occs_debt_wo_dups['Occupation'] == occ_2].index, inplace=True) 
                occ_list.remove(occ_2)

occs_debt_wo_dups = occs_debt_wo_dups.reset_index(drop=True) #reset indexes 0-65 

#take the first two words of every job if applicable and remove any duplicate titles. handles longer titles 
two_word_occs = {}
#retrieve all two worded occupations
for occ in occ_list:
    if len(occ.split()) >= 2: #only account for words that are actually more than two words long
        occ_two_word = occ.split()[0] + " " + occ.split()[1] #get first two words
        two_word_occs[occ_list.index(occ)] = occ_two_word #append both words to dict

#retrieve any duplicates now 
flipped = {} 
for key, value in two_word_occs.items():
    flipped.setdefault(value, list()).append(key) #flip keys with values

keys_w_dups = [key for key, values in flipped.items() if len(values) > 1] #retrieve keys that have multiple values. store them in a list

print(occurences_dict)

{'Farmer': 75, 'Esquire': 70, 'Widow': 37, 'Esq': 9, 'Inn Keeper': 8, 'Druggist': 6, 'Maiden': 4, 'Blacksmith': 5, 'County': 3, 'Counseller at Low': 20, 'Merchants': 55, 'Turner': 3, 'Barker': 17, 'Clerk': 3, 'Senator ': 3, 'Executor Rebecca Forman': 2, 'Physian': 12, 'Hunterdon County': 2, 'Late Sheriff': 2, 'Fanner': 2, 'Stage Driver': 2, 'Late of Trenton': 2, 'Schoolmaster': 5, 'Mason': 2, 'Speaker of Assembly': 2, 'Practitioner in Physick': 2, 'Attorney at law': 10, 'Attorney General': 2, 'Hatter': 2, 'Now Peasson, Wife of Job Peasson': 2, 'Cooper': 2, 'Brewer': 2, 'Executor of Thomas Newman': 1, 'Gentleman': 1, 'Treasurer': 1, 'Smalley of Cumberland': 1, 'Adm of John Holmes Deceased': 1, 'Executor of John Budd Deceased': 1, 'Treasurer & Co': 1, 'Jurviving Executor of John Leidt Deceased': 1, 'Executor of Jacob Van Dome': 1, 'Trenton': 1, 'Guardian of Jane Williamson': 1, 'Guardian of Abram Tenbrock': 1, 'Administrators of James Chambers Deceased': 1, 'Wider': 3, 'Wife of Samuel Bo

In [562]:
print(occs_debt_wo_dups.to_string())

                                     Occupation  Dollars   Cents
0                              (Son of Timothy)    252.0   158.0
1                   Adm of John Holmes Deceased      0.0     0.0
2                Adm of William Bunnet Deceased      3.0    87.0
3             Admin of James McMichael Deceased    264.0   194.0
4        Administrator of John Connets deceased     89.0    25.0
5     Administrators of James Chambers Deceased    125.0    69.0
6                                        At Law     49.0    50.0
7                              Attorney General   3885.0   108.0
8                               Attorney at law    514.0   576.0
9                                        Barker   5438.0   701.0
10                                   Blacksmith    546.0   215.0
11                                       Brewer    341.0    78.0
12                                    Carpinter    404.0   164.0
13                                        Clerk     57.0   156.0
14                       

In [563]:
print(flipped)
print("\n\n")
print(keys_w_dups)

{'(Son of': [0], 'Adm of': [1, 2], 'Admin of': [3], 'Administrator of': [4], 'Administrators of': [5], 'At Law': [6], 'Attorney General': [7], 'Attorney at': [8], 'Columbea College': [14], 'Counseller at': [16], 'Esquire Treasurer': [22], 'Executer of': [23], 'Executor Rebecca': [24], 'Executor of': [25, 26, 27, 28], 'Executors of': [29], 'Guardian of': [33, 34, 35, 36, 37], 'Hunterdon County': [39], 'Inn Keeper': [40], 'Jurviving Executor': [41], 'Late Sheriff': [42], 'Late of': [43], 'Moridenhead Meadour': [47], 'Now Peasson,': [48], 'Practitioner in': [50], 'Smalley of': [53], 'Somerset County': [54], 'Speaker of': [55], 'Stage Driver': [56], 'Treasurer &': [60], 'Trust &': [62], 'Trust for': [63], 'Wife of': [68]}



['Adm of', 'Executor of', 'Guardian of']


In [564]:
print(occurences_dict)

for occ in keys_w_dups:
    i = 0
    while i < len(flipped[occ]):
        index = flipped[occ][i]
        if len(flipped[occ]) > 1: #make sure to keep one index in the list. don't want to delete ALL occurences
            duplicate_row = occs_debt_wo_dups.loc[[index]] #return row that has index
            dollars_in_dup_row = duplicate_row[['Dollars']].sum(axis=1).values #sum all dollars in row                
            cents_in_dup_row = duplicate_row[['Cents']].sum(axis=1).values #return total cents in row
            dollars_dup_row = dollars_in_dup_row[0] #stored in array. need to retrieve value
            cents_dup_row = cents_in_dup_row[0] #stored in array. need to retrieve value

            last_index = flipped[occ][len(flipped[occ]) - 1] #get last value in list. we will add dollars and cents here
                        
            occs_debt_wo_dups.loc[last_index, 'Dollars'] += dollars_dup_row #add dollars from one occupation to the other
            occs_debt_wo_dups.loc[last_index, 'Cents'] += cents_dup_row #add cents from one occupation to the other
            
            #remove value from occurences dictionary and add one to other occupation
            
            print(occs_debt_wo_dups.loc[last_index, 'Occupation'])
            occurences_dict[occs_debt_wo_dups.loc[last_index, 'Occupation']] += occurences_dict[occs_debt_wo_dups.loc[index, 'Occupation']] 
            occurences_dict.pop(occs_debt_wo_dups.loc[index, 'Occupation'])
            
            print("Length: " + str(len(flipped[occ])))
            print("Last Index: " + str(last_index) + " Index: " + str(index))
            print(flipped[occ])
            print("\n\n")
                        
            occs_debt_wo_dups.drop(index, inplace=True) #remove duplicate occupation now
            flipped[occ].remove(index) #remove from list
            
        else:
            i += 1

occs_debt_wo_dups = occs_debt_wo_dups.reset_index(drop=True) #reset indexes 0-...

{'Farmer': 75, 'Esquire': 70, 'Widow': 37, 'Esq': 9, 'Inn Keeper': 8, 'Druggist': 6, 'Maiden': 4, 'Blacksmith': 5, 'County': 3, 'Counseller at Low': 20, 'Merchants': 55, 'Turner': 3, 'Barker': 17, 'Clerk': 3, 'Senator ': 3, 'Executor Rebecca Forman': 2, 'Physian': 12, 'Hunterdon County': 2, 'Late Sheriff': 2, 'Fanner': 2, 'Stage Driver': 2, 'Late of Trenton': 2, 'Schoolmaster': 5, 'Mason': 2, 'Speaker of Assembly': 2, 'Practitioner in Physick': 2, 'Attorney at law': 10, 'Attorney General': 2, 'Hatter': 2, 'Now Peasson, Wife of Job Peasson': 2, 'Cooper': 2, 'Brewer': 2, 'Executor of Thomas Newman': 1, 'Gentleman': 1, 'Treasurer': 1, 'Smalley of Cumberland': 1, 'Adm of John Holmes Deceased': 1, 'Executor of John Budd Deceased': 1, 'Treasurer & Co': 1, 'Jurviving Executor of John Leidt Deceased': 1, 'Executor of Jacob Van Dome': 1, 'Trenton': 1, 'Guardian of Jane Williamson': 1, 'Guardian of Abram Tenbrock': 1, 'Administrators of James Chambers Deceased': 1, 'Wider': 3, 'Wife of Samuel Bo

In [565]:
print(occs_debt_wo_dups.to_string())

                                     Occupation  Dollars   Cents
0                              (Son of Timothy)    252.0   158.0
1                Adm of William Bunnet Deceased      3.0    87.0
2             Admin of James McMichael Deceased    264.0   194.0
3        Administrator of John Connets deceased     89.0    25.0
4     Administrators of James Chambers Deceased    125.0    69.0
5                                        At Law     49.0    50.0
6                              Attorney General   3885.0   108.0
7                               Attorney at law    514.0   576.0
8                                        Barker   5438.0   701.0
9                                    Blacksmith    546.0   215.0
10                                       Brewer    341.0    78.0
11                                    Carpinter    404.0   164.0
12                                        Clerk     57.0   156.0
13                             Columbea College     62.0    24.0
14                       

In [566]:
exceptions = ['Attorney General', 'Hunterdon County', 'Inn Keeper',
             'Late Sheriff', 'Late of Trenton', 'Now Peasson, Wife of Job Peasson', 'Stage Driver'] #applicable only to a few states

In [567]:
#rename occupations, if necessary
def rename(occ):
    if 'Now Peasson' in occ:
        flipped['Wife'] = flipped.pop('Now Peasson,')
        occurences_dict['Wife'] = occurences_dict.pop('Now Peasson, Wife of Job Peasson')
        return 'Wife'
    else:
        return occ
    
occs_debt_wo_dups['Occupation'] = occs_debt_wo_dups['Occupation'].apply(rename)
print(occs_debt_wo_dups.to_string())

                                     Occupation  Dollars   Cents
0                              (Son of Timothy)    252.0   158.0
1                Adm of William Bunnet Deceased      3.0    87.0
2             Admin of James McMichael Deceased    264.0   194.0
3        Administrator of John Connets deceased     89.0    25.0
4     Administrators of James Chambers Deceased    125.0    69.0
5                                        At Law     49.0    50.0
6                              Attorney General   3885.0   108.0
7                               Attorney at law    514.0   576.0
8                                        Barker   5438.0   701.0
9                                    Blacksmith    546.0   215.0
10                                       Brewer    341.0    78.0
11                                    Carpinter    404.0   164.0
12                                        Clerk     57.0   156.0
13                             Columbea College     62.0    24.0
14                       

In [568]:
print(flipped)

{'(Son of': [0], 'Adm of': [2], 'Admin of': [3], 'Administrator of': [4], 'Administrators of': [5], 'At Law': [6], 'Attorney General': [7], 'Attorney at': [8], 'Columbea College': [14], 'Counseller at': [16], 'Esquire Treasurer': [22], 'Executer of': [23], 'Executor Rebecca': [24], 'Executor of': [28], 'Executors of': [29], 'Guardian of': [37], 'Hunterdon County': [39], 'Inn Keeper': [40], 'Jurviving Executor': [41], 'Late Sheriff': [42], 'Late of': [43], 'Moridenhead Meadour': [47], 'Practitioner in': [50], 'Smalley of': [53], 'Somerset County': [54], 'Speaker of': [55], 'Stage Driver': [56], 'Treasurer &': [60], 'Trust &': [62], 'Trust for': [63], 'Wife of': [68], 'Wife': [48]}


In [569]:
print(occurences_dict)

{'Farmer': 75, 'Esquire': 70, 'Widow': 37, 'Esq': 9, 'Inn Keeper': 8, 'Druggist': 6, 'Maiden': 4, 'Blacksmith': 5, 'County': 3, 'Counseller at Low': 20, 'Merchants': 55, 'Turner': 3, 'Barker': 17, 'Clerk': 3, 'Senator ': 3, 'Executor Rebecca Forman': 2, 'Physian': 12, 'Hunterdon County': 2, 'Late Sheriff': 2, 'Fanner': 2, 'Stage Driver': 2, 'Late of Trenton': 2, 'Schoolmaster': 5, 'Mason': 2, 'Speaker of Assembly': 2, 'Practitioner in Physick': 2, 'Attorney at law': 10, 'Attorney General': 2, 'Hatter': 2, 'Cooper': 2, 'Brewer': 2, 'Executor of Thomas Newman': 5, 'Gentleman': 1, 'Treasurer': 1, 'Smalley of Cumberland': 1, 'Treasurer & Co': 1, 'Jurviving Executor of John Leidt Deceased': 1, 'Trenton': 1, 'Administrators of James Chambers Deceased': 1, 'Wider': 3, 'Wife of Samuel Bonnall': 1, 'Trust & co.': 1, 'Esquire Treasurer and Co': 1, 'Guardian of Sons of George Green  Deceased': 5, 'Columbea College': 1, 'Wheelright': 2, 'Elder': 1, 'Executer of Cornelius Vanhorne': 2, 'Executors o

In [570]:
#get the first word from every occupation and store any duplicates in a dictionary
occ_list = occs_debt_wo_dups['Occupation'].to_list()
occupations = []
one_word_occs = {}
flipped_one_w = {}

for occ in occ_list:
    if occ not in exceptions:
        first_word = occ.split()[0]
        one_word_occs[occ_list.index(occ)] = first_word 
        
#flip dictionary
for key, value in one_word_occs.items():
    flipped_one_w.setdefault(value, list()).append(key) #flip keys with values

print(flipped_one_w)

{'(Son': [0], 'Adm': [1], 'Admin': [2], 'Administrator': [3], 'Administrators': [4], 'At': [5], 'Attorney': [7], 'Barker': [8], 'Blacksmith': [9], 'Brewer': [10], 'Carpinter': [11], 'Clerk': [12], 'Columbea': [13], 'Cooper': [14], 'Counseller': [15], 'County': [16], 'Druggist': [17], 'Elder': [18], 'Esq': [19], 'Esquire': [20, 21], 'Executer': [22], 'Executor': [23, 24], 'Executors': [25], 'Fanner': [26], 'Farmer': [27], 'Gentleman': [28], 'Guardian': [29], 'Hatter': [30], 'Jurviving': [33], 'Maiden': [36], 'Mason': [37], 'Merchants': [38], 'Moridenhead': [39], 'Wife': [40, 60], 'Physian': [41], 'Practitioner': [42], 'Schoolmaster': [43], 'Senator': [44], 'Smalley': [45], 'Somerset': [46], 'Speaker': [47], 'Tamer': [49], 'Tayler': [50], 'Treasurer': [51, 52], 'Trenton': [53], 'Trust': [54, 55], 'Turner': [56], 'Wheelright': [57], 'Wider': [58], 'Widow': [59]}


In [571]:
#use fuzzy string matching 
flipped_list = list(flipped_one_w) #convert to list. needed in order to iterate through
print(flipped_list)
print("\n")
for occ in flipped_list:
    for occ_2 in flipped_list:
        if (occ_2 != occ and rapidfuzz.fuzz.ratio(occ, occ_2) >= 80 or (occ == 'Adm' and occ_2 == 'Admin') or (occ == 'Administrator' and occ_2 == 'Adm') 
            or (occ == 'Jurviving' and occ_2 == 'Executors') or (occ == 'Esq' and occ_2 == 'Esquire') or (occ == 'Attorney' and occ_2 == 'At')):
            flipped_one_w[occ].extend(flipped_one_w[occ_2]) #combine lists
            flipped_one_w.pop(occ_2) #remove second occurence of the same occupation 
            flipped_list.remove(occ_2) #remove from list 
            
keys_w_dups = [key for key, values in flipped_one_w.items() if len(values) > 1] #retrieve keys that have multiple values. store them in a list
         
print(flipped_one_w)
print("\n\n")
print(keys_w_dups)

['(Son', 'Adm', 'Admin', 'Administrator', 'Administrators', 'At', 'Attorney', 'Barker', 'Blacksmith', 'Brewer', 'Carpinter', 'Clerk', 'Columbea', 'Cooper', 'Counseller', 'County', 'Druggist', 'Elder', 'Esq', 'Esquire', 'Executer', 'Executor', 'Executors', 'Fanner', 'Farmer', 'Gentleman', 'Guardian', 'Hatter', 'Jurviving', 'Maiden', 'Mason', 'Merchants', 'Moridenhead', 'Wife', 'Physian', 'Practitioner', 'Schoolmaster', 'Senator', 'Smalley', 'Somerset', 'Speaker', 'Tamer', 'Tayler', 'Treasurer', 'Trenton', 'Trust', 'Turner', 'Wheelright', 'Wider', 'Widow']


{'(Son': [0], 'Administrator': [3, 1, 2, 4], 'Attorney': [7, 5], 'Barker': [8], 'Blacksmith': [9], 'Brewer': [10], 'Carpinter': [11], 'Clerk': [12], 'Columbea': [13], 'Cooper': [14], 'Counseller': [15], 'County': [16], 'Druggist': [17], 'Elder': [18], 'Esq': [19, 20, 21], 'Fanner': [26], 'Farmer': [27], 'Gentleman': [28], 'Guardian': [29], 'Hatter': [30], 'Jurviving': [33, 25, 22, 23, 24], 'Maiden': [36], 'Mason': [37], 'Merchants': 

In [572]:
#remove duplicates
for occ in keys_w_dups:
    i = 0
    while i < len(flipped_one_w[occ]):
        if len(flipped_one_w[occ]) > 1:
            index = flipped_one_w[occ][i]
            last_index = flipped_one_w[occ][len(flipped_one_w[occ]) - 1] #get last value in list. we will add dollars and cents here
            duplicate_row = occs_debt_wo_dups.loc[[index]] #returns row that has index
            dollars_in_dup_row = duplicate_row[['Dollars']].sum(axis=1).values[0]
            cents_in_dup_row = duplicate_row[['Cents']].sum(axis=1).values[0]
            
            occs_debt_wo_dups.loc[last_index, 'Dollars'] += dollars_in_dup_row
            occs_debt_wo_dups.loc[last_index, 'Cents'] += cents_dup_row
            
            occurences_dict[occs_debt_wo_dups.loc[last_index, 'Occupation']] += occurences_dict[occs_debt_wo_dups.loc[index, 'Occupation']] 
            occurences_dict.pop(occs_debt_wo_dups.loc[index, 'Occupation'])

            occs_debt_wo_dups.drop(index, inplace=True)
            flipped_one_w[occ].remove(index)
            
        else:
            i += 1

occs_debt_wo_dups = occs_debt_wo_dups.reset_index(drop=True) #reset indexes 0-...

print(occs_debt_wo_dups.to_string())

                                     Occupation  Dollars   Cents
0                              (Son of Timothy)    252.0   158.0
1     Administrators of James Chambers Deceased    481.0   354.0
2                                        At Law    563.0   145.0
3                              Attorney General   3885.0   108.0
4                                        Barker   5438.0   701.0
5                                    Blacksmith    546.0   215.0
6                                        Brewer    341.0    78.0
7                                     Carpinter    404.0   164.0
8                                         Clerk     57.0   156.0
9                              Columbea College     62.0    24.0
10                                       Cooper    173.0   138.0
11                            Counseller at Low  13619.0   881.0
12                                       County    583.0   127.0
13                                     Druggist    328.0   336.0
14                       

In [573]:
occs_debt_wo_dups[['Cents']] = occs_debt_wo_dups[['Cents']].div(100)
print(occs_debt_wo_dups.to_string())

                                     Occupation  Dollars  Cents
0                              (Son of Timothy)    252.0   1.58
1     Administrators of James Chambers Deceased    481.0   3.54
2                                        At Law    563.0   1.45
3                              Attorney General   3885.0   1.08
4                                        Barker   5438.0   7.01
5                                    Blacksmith    546.0   2.15
6                                        Brewer    341.0   0.78
7                                     Carpinter    404.0   1.64
8                                         Clerk     57.0   1.56
9                              Columbea College     62.0   0.24
10                                       Cooper    173.0   1.38
11                            Counseller at Low  13619.0   8.81
12                                       County    583.0   1.27
13                                     Druggist    328.0   3.36
14                                      

In [574]:
occs_debt_wo_dups['Total'] = occs_debt_wo_dups[['Dollars', 'Cents']].sum(axis=1) #add totals to new column
print(occs_debt_wo_dups.to_string())

                                     Occupation  Dollars  Cents     Total
0                              (Son of Timothy)    252.0   1.58    253.58
1     Administrators of James Chambers Deceased    481.0   3.54    484.54
2                                        At Law    563.0   1.45    564.45
3                              Attorney General   3885.0   1.08   3886.08
4                                        Barker   5438.0   7.01   5445.01
5                                    Blacksmith    546.0   2.15    548.15
6                                        Brewer    341.0   0.78    341.78
7                                     Carpinter    404.0   1.64    405.64
8                                         Clerk     57.0   1.56     58.56
9                              Columbea College     62.0   0.24     62.24
10                                       Cooper    173.0   1.38    174.38
11                            Counseller at Low  13619.0   8.81  13627.81
12                                    

In [575]:
occs_debt_wo_dups['# of People'] = occs_debt_wo_dups['Occupation'].map(occurences_dict) #add occurences to table
print(occs_debt_wo_dups[['Occupation', '# of People']])

                                     Occupation  # of People
0                              (Son of Timothy)            2
1     Administrators of James Chambers Deceased            6
2                                        At Law           11
3                              Attorney General            2
4                                        Barker           17
5                                    Blacksmith            5
6                                        Brewer            2
7                                     Carpinter            4
8                                         Clerk            3
9                              Columbea College            1
10                                       Cooper            2
11                            Counseller at Low           20
12                                       County            3
13                                     Druggist            6
14                                        Elder            1
15                     E

In [576]:
occs_debt_wo_dups['Average'] = occs_debt_wo_dups['Total'] / occs_debt_wo_dups['# of People'] #calculate average and add to new column
print(occs_debt_wo_dups[['Occupation', 'Total', '# of People', 'Average']].to_string())

                                     Occupation     Total  # of People      Average
0                              (Son of Timothy)    253.58            2   126.790000
1     Administrators of James Chambers Deceased    484.54            6    80.756667
2                                        At Law    564.45           11    51.313636
3                              Attorney General   3886.08            2  1943.040000
4                                        Barker   5445.01           17   320.294706
5                                    Blacksmith    548.15            5   109.630000
6                                        Brewer    341.78            2   170.890000
7                                     Carpinter    405.64            4   101.410000
8                                         Clerk     58.56            3    19.520000
9                              Columbea College     62.24            1    62.240000
10                                       Cooper    174.38            2    87

In [577]:
exceptions = ['Attorney General', 'Columbea College', 'Hunterdon County', 'Inn Keeper', 'Late Sheriff',
             'Late of Trenton', 'Moridenhead Meadour', 'Somerset County', 'Stage Driver'] #applicable only to a few states

In [578]:
#convert to one word
def first_word_only(occ):
    if occ not in exceptions:
        return occ.split()[0]
    else:
        return occ
    
occs_debt_wo_dups['Occupation'] = occs_debt_wo_dups['Occupation'].apply(first_word_only)
print(occs_debt_wo_dups.to_string())

             Occupation  Dollars  Cents     Total  # of People      Average
0                  (Son    252.0   1.58    253.58            2   126.790000
1        Administrators    481.0   3.54    484.54            6    80.756667
2                    At    563.0   1.45    564.45           11    51.313636
3      Attorney General   3885.0   1.08   3886.08            2  1943.040000
4                Barker   5438.0   7.01   5445.01           17   320.294706
5            Blacksmith    546.0   2.15    548.15            5   109.630000
6                Brewer    341.0   0.78    341.78            2   170.890000
7             Carpinter    404.0   1.64    405.64            4   101.410000
8                 Clerk     57.0   1.56     58.56            3    19.520000
9      Columbea College     62.0   0.24     62.24            1    62.240000
10               Cooper    173.0   1.38    174.38            2    87.190000
11           Counseller  13619.0   8.81  13627.81           20   681.390500
12          

In [579]:
occs_debt_wo_dups.loc[0, 'Occupation'] = 'Son'
occs_debt_wo_dups.loc[2, 'Occupation'] = 'Attorney'
print(occs_debt_wo_dups.to_string())

             Occupation  Dollars  Cents     Total  # of People      Average
0                   Son    252.0   1.58    253.58            2   126.790000
1        Administrators    481.0   3.54    484.54            6    80.756667
2              Attorney    563.0   1.45    564.45           11    51.313636
3      Attorney General   3885.0   1.08   3886.08            2  1943.040000
4                Barker   5438.0   7.01   5445.01           17   320.294706
5            Blacksmith    546.0   2.15    548.15            5   109.630000
6                Brewer    341.0   0.78    341.78            2   170.890000
7             Carpinter    404.0   1.64    405.64            4   101.410000
8                 Clerk     57.0   1.56     58.56            3    19.520000
9      Columbea College     62.0   0.24     62.24            1    62.240000
10               Cooper    173.0   1.38    174.38            2    87.190000
11           Counseller  13619.0   8.81  13627.81           20   681.390500
12          

In [580]:
total = occs_debt_wo_dups['Total'].sum()
occs_debt_wo_dups['% of Total Debt'] = round((occs_debt_wo_dups['Total'] / total) * 100, 3)
print(occs_debt_wo_dups[['Occupation', 'Total', '# of People', 'Average', '% of Total Debt']].to_string())

             Occupation     Total  # of People      Average  % of Total Debt
0                   Son    253.58            2   126.790000            0.130
1        Administrators    484.54            6    80.756667            0.249
2              Attorney    564.45           11    51.313636            0.290
3      Attorney General   3886.08            2  1943.040000            1.994
4                Barker   5445.01           17   320.294706            2.795
5            Blacksmith    548.15            5   109.630000            0.281
6                Brewer    341.78            2   170.890000            0.175
7             Carpinter    405.64            4   101.410000            0.208
8                 Clerk     58.56            3    19.520000            0.030
9      Columbea College     62.24            1    62.240000            0.032
10               Cooper    174.38            2    87.190000            0.089
11           Counseller  13627.81           20   681.390500            6.994

In [581]:
#final total debt held by people w/ no job listed
occs_and_debt = pd.read_excel(path,
                             header=11, usecols='G, L, M') #occupations plus continental debt for conneticut
occs_and_debt = occs_and_debt.rename(columns={'Unnamed: 6': 'Occupation'})
no_occ = occs_and_debt[occs_and_debt['Occupation'].isna()]

num_of_people = len(no_occ)
no_occ = no_occ.reset_index(drop=True)
no_occ[['Cents']] = no_occ[['Cents']].div(100)
total_series = no_occ[['Dollars']].sum(axis=1)
total = total_series.sum()

occs_debt_wo_dups = occs_debt_wo_dups.append({'Occupation': 'None Listed', 'Total': total, '# of People': num_of_people, 'Average': total / num_of_people}, ignore_index=True)
print(occs_debt_wo_dups[['Occupation', 'Total', '# of People', 'Average', '% of Total Debt']].to_string())

             Occupation      Total  # of People      Average  % of Total Debt
0                   Son     253.58            2   126.790000            0.130
1        Administrators     484.54            6    80.756667            0.249
2              Attorney     564.45           11    51.313636            0.290
3      Attorney General    3886.08            2  1943.040000            1.994
4                Barker    5445.01           17   320.294706            2.795
5            Blacksmith     548.15            5   109.630000            0.281
6                Brewer     341.78            2   170.890000            0.175
7             Carpinter     405.64            4   101.410000            0.208
8                 Clerk      58.56            3    19.520000            0.030
9      Columbea College      62.24            1    62.240000            0.032
10               Cooper     174.38            2    87.190000            0.089
11           Counseller   13627.81           20   681.390500    

In [582]:
occs_debt_wo_dups[['Occupation', 'Total', '# of People', 'Average', '% of Total Debt']].to_excel("./FinishedSpreadsheets/occupation_debt_" + state + ".xlsx")